In [1]:
import pipeline.pipeline as pipeline
import db.knowhere_db as kdb
import time
import pandas as pd

In [ ]:
fName = r'C:\Users\William\Documents\DataScience\Projects\Knowhere\data\aggregated_emil_sensor2.csv'
csv_dir = r'C:\Users\William\Documents\DataScience\Projects\Knowhere\data\iPhone'

In [ ]:
user_df = pipeline.iphone(username='emil', file_with_path=fName, dir_to_aggregate=csv_dir)

In [ ]:
#writer = kdb.Writer('knowhere')
#writer.write_dataframe_to_collection(df=user_df, collection='iphone_test2')

In [ ]:
reader = kdb.Reader('knowhere')
rdf = reader.get_dataframe_unrolled(collection='iphone_test',username='emil',sensor='GPS')
rdf[0:10]

In [2]:
reader = kdb.Reader('knowhere')
rdf = reader.get_dataframe_pivoted(collection='iphone_test',sensor='GPS')
rdf[0:10]

db\knowhere_db.py:74: Warning: Excluding user_id from filter can cause errors during pivot
  warnings.warn('Excluding user_id from filter can cause errors during pivot', Warning)


ValueError: Index contains duplicate entries, cannot reshape